#### Fidel C. Otero

#### Student UT EID: fco229

#### Date Created: 10-03-2023

#### Date Last Modified: 10-03-2023

In [1]:
# import libraries
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

##### 1. Read the Yale Bright Star Catalog and create a Pandas data frame object.

In [2]:
df = pd.read_csv('bsc5.csv', skipinitialspace=True)

In [3]:
df.columns

Index(['HR', 'Name', 'DM', 'HD', 'SAO', 'FK5', 'IRflag', 'r_IRflag',
       'Multiple', 'ADS', 'ADScomp', 'VarID', 'RAh1900', 'RAm1900', 'RAs1900',
       'DE-1900', 'DEd1900', 'DEm1900', 'DEs1900', 'RAh', 'RAm', 'RAs', 'DE-',
       'DEd', 'DEm', 'DEs', 'GLON', 'GLAT', 'Vmag', 'n_Vmag', 'u_Vmag', 'B-V',
       'u_B-V', 'U-B', 'u_U-B', 'R-I', 'n_R-I', 'SpType', 'n_SpType', 'pmRA',
       'pmDE', 'n_Parallax', 'Parallax', 'RadVel', 'n_RadVel', 'l_RotVel',
       'RotVel', 'u_RotVel', 'Dmag', 'Sep', 'MultID', 'MultCnt', 'NoteFlag'],
      dtype='object')

In [4]:
len(df)

13605

##### 2. Drop all objects that do not have a trigonometric parallax.

In [5]:
df = df.dropna(subset='Parallax')

##### 3. Create a column called Distance that has the distance of the object in parsec. Remember that the inverse of the parallax in arc_seconds is the distance in parsec.

In [6]:
df['Distance'] = 1 / df['Parallax']

In [7]:
df['Distance']

3         71.428571
6         21.276596
7         20.000000
9         14.925373
18        31.250000
            ...    
13575    100.000000
13581    142.857143
13591     62.500000
13593     76.923077
13598     28.571429
Name: Distance, Length: 3289, dtype: float64

##### 4. Use the RA and Dec for the J2000.0 epoch for the next set of computations.

In [8]:
df['ra'] = (df['RAh'] + (df['RAm'] / 60) + (df['RAs'] / 3600)) * 15

In [9]:
df['DE-'].replace(to_replace=['+', '-'], value=[1, -1], inplace=True)
df['dec'] = ((df['DEd'] + (df['DEm'] / 60) + (df['DEs'] / 3600)) * df['DE-'])

In [10]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.coordinates
c = SkyCoord(ra=df['ra']*u.degree, dec=df['dec']*u.degree)

ra = c.ra.wrap_at(180 * u.deg).radian
dec = c.dec.radian

long = c.galactic.l.wrap_at(180 * u.deg).radian
lat = c.galactic.b.radian
r = df['Distance']

In [11]:
r = r * math.pi / 180
r = np.array(r)

In [12]:
len(r)

3289

In [13]:
len(lat)

3289

In [14]:
len(long)

3289

##### 5.  Create three columns x, y, and z that will have the distances to the stars in parsecs in the equatorial frame of reference. Write a routine that will do a coordinate transformation from a spherical coordinate system to a Cartesian system. In this Cartesian system, the x-axis is pointing to the vernal equinox and the x-y plane is the plane of the celestial equator. The z-axis should be pointing to the north celestial pole. You can use the library Astropy for your work.

In [15]:
df['x'], df['y'], df['z'] = astropy.coordinates.spherical_to_cartesian(r, lat, long)

In [16]:
df['x']

3       -0.033287
6       -0.168395
7        0.109561
9       -0.079381
18      -0.169672
           ...   
13575   -0.817850
13581    0.194954
13591    0.019022
13593   -0.636590
13598   -0.167369
Name: x, Length: 3289, dtype: float64

##### 6. Create a column called Color in your data frame. The colors could be named colors like red or blue. Or, the colors could be in hexadecimal format. Here is the color scheme according to spectral types - O (deepest blue), B (medium blue), A (light blue), F (green), G (yellow), K (orange), M (red), and any other star not having a spectral type black.

In [17]:
colors_dict = {'O': '#7F00FF', 'B': '#0000cd', 'A': '#ADD8E6', 'F': '#008000',
              'G': '#FFFF00', 'K': '#FFA500', 'M': '#ff0000'}

df['colors'] = ''
for ind in df['SpType'].loc[df['SpType'].isna() == False].index:
    for char in df['SpType'][ind]:
        if char.isupper():
            df['colors'][ind] = char
            break
df['colors_hex'] = ''

for ind in df.index:
    if df['colors'][ind] in colors_dict.keys():
        df['colors_hex'][ind] = colors_dict[df['colors'][ind]]
    else:
        df['colors_hex'][ind] = '#000000'

/var/folders/2l/5z_t54nn2mgcnb_rlc4qxcr40000gn/T/ipykernel_54041/3205642460.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['colors'][ind] = char
/var/folders/2l/5z_t54nn2mgcnb_rlc4qxcr40000gn/T/ipykernel_54041/3205642460.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['colors_hex'][ind] = colors_dict[df['colors'][ind]]
/var/folders/2l/5z_t54nn2mgcnb_rlc4qxcr40000gn/T/ipykernel_54041/3205642460.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [18]:
df.rename(columns={'colors_hex': 'Color'}, inplace=True)
df.drop('colors', axis=1, inplace=True)

In [19]:
df

,HR,Name,DM,HD,SAO,FK5,IRflag,r_IRflag,Multiple,ADS,...,MultID,MultCnt,NoteFlag,Distance,ra,dec,x,y,z,Color
3,3.0,33 Psc,BD-06 6357,28.0,128572.0,1002.0,I,NaN,NaN,NaN,...,NaN,3.0,*,71.428571,1.333750,-5.707500,-0.033287,0.507313,-1.138286,#FFA500
6,5.0,NaN,BD+57 2865,123.0,21085.0,NaN,NaN,NaN,NaN,61.0,...,NaN,NaN,*,21.276596,1.566667,58.436667,-0.168395,0.329997,-0.025367,#FFFF00
7,6.0,NaN,CD-4914337,142.0,214963.0,NaN,NaN,NaN,W,NaN,...,NaN,NaN,*,20.000000,1.579167,-49.075000,0.109561,-0.086873,-0.319838,#FFFF00
9,8.0,NaN,BD+28 4704,166.0,73743.0,NaN,NaN,NaN,NaN,69.0,...,AB,4.0,*,14.925373,1.653333,29.021389,-0.079381,0.203983,-0.141238,#FFA500
18,15.0,21Alp And,BD+28 4,358.0,73765.0,1.0,I,NaN,NaN,94.0,...,NaN,NaN,*,31.250000,2.097083,29.090556,-0.169672,0.425665,-0.295802,#0000cd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,9094.0,NaN,BD+65 1987,225009.0,10937.0,NaN,NaN,NaN,NaN,1.0,...,AB,3.0,*,100.000000,0.650417,66.098889,-0.817850,1.537739,0.112487,#FFFF00
13581,9098.0,2 Cet,BD-18 6417,225132.0,147059.0,905.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,142.857143,0.935000,-17.336111,0.194954,0.603606,-2.411293,#0000cd
13591,9103.0,3 Cet,BD-11 6194,225212.0,147066.0,2001.0,I,NaN,NaN,NaN,...,NaN,NaN,NaN,62.500000,1.125417,-10.509444,0.019022,0.371852,-1.025317,#FFA500
13593,9104.0,NaN,BD+66 1679,225216.0,10956.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,76.923077,1.175000,67.166667,-0.636590,1.176898,0.110153,#FFA500


##### 7. Use Plotly to obtain the 3-D distribution of the stars.
##### 8. The stars should be color-coded.
##### 9. If you hover over a star, you should get the following information - HR Number, Distance in parsec, and Radial Velocity in km/s.  

In [26]:
import plotly.express as px
fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='Color', hover_name='HR', hover_data=['Distance', 'RadVel'], title='3D Map of the Universe')
fig.show()